In [2]:
import pandas as pd
import numpy as np
import os
import requests

In [ ]:
flights_urls = ['http://stat-computing.org/dataexpo/2009/2004.csv.bz2',
'http://stat-computing.org/dataexpo/2009/2005.csv.bz2',
'http://stat-computing.org/dataexpo/2009/2006.csv.bz2',
'http://stat-computing.org/dataexpo/2009/2007.csv.bz2',
'http://stat-computing.org/dataexpo/2009/2008.csv.bz2']

In [ ]:
# download the dataset with pandas
flights_bz2 = 'flight_data_bz2'
#if not os.path.exists(flights_bz2):
#    os.makedirs(flights_bz2)

#for url in flights_urls:
#    response = requests.get(url)
#    with open (os.path.join(flights_bz2, url.split('/')[-1]), mode = 'wb') as file:
#        file.write(response.content)

## Gathering

In [7]:
df_2004= pd.read_csv('data/2004.csv', nrows=None, encoding='latin-1')
df_2004.shape

(7129270, 29)

In [4]:
df_2005= pd.read_csv('data/2005.csv', nrows=None, encoding='latin-1')
df_2005.shape

(7140596, 29)

In [5]:
df_2006= pd.read_csv('data/2006.csv', nrows=None, encoding='latin-1')
df_2006.shape

(7141922, 29)

In [3]:
df_2007= pd.read_csv('data/2007.csv', nrows=None, encoding='latin-1')
df_2007.shape

(7453215, 29)

In [9]:
df_2008= pd.read_csv('data/2008.csv', nrows=None, encoding='latin-1')
df_2008.shape

(7009728, 29)

In [10]:
df = pd.concat([df_2004, df_2005, df_2006, df_2007, df_2008])

In [11]:
df.shape

(35874731, 29)

In [12]:
df.sample(10)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
7090458,2005,12,3,6,655.0,700,952.0,958,CO,239,...,8.0,13.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
6726173,2006,12,18,1,1927.0,1928,2026.0,2021,OO,5707,...,3.0,10.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
2759531,2008,5,27,2,1349.0,1341,1555.0,1555,FL,925,...,6.0,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1745909,2004,4,16,5,1155.0,1200,1250.0,1307,UA,615,...,5.0,20.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
6428219,2004,11,9,2,1641.0,1646,1833.0,1840,AA,2219,...,9.0,21.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
3500768,2008,6,3,2,1845.0,1855,2130.0,2145,AA,1329,...,5.0,31.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
5406524,2004,10,5,2,1113.0,1105,1304.0,1300,WN,1701,...,4.0,7.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
6155277,2008,11,18,2,1406.0,1415,1711.0,1727,UA,1426,...,5.0,9.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
6912365,2008,12,4,4,1838.0,1835,2232.0,2235,AA,892,...,5.0,13.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
6895224,2008,12,29,1,1716.0,1700,1846.0,1843,9E,5857,...,8.0,11.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN


## Assessing

In [13]:
df.head(30).T

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
Year,2004,2004,2004,2004,2004,2004,2004,2004,2004,2004,...,2004,2004,2004,2004,2004,2004,2004,2004,2004,2004
Month,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
DayofMonth,12,13,14,15,16,17,19,20,21,22,...,3,4,5,6,7,8,9,10,11,12
DayOfWeek,1,2,3,4,5,6,1,2,3,4,...,6,7,1,2,3,4,5,6,7,1
DepTime,623,621,633,627,635,628,650,627,623,626,...,1523,NaN,1622,NaN,NaN,NaN,1509,1554,1513,1511
CRSDepTime,630,630,630,630,630,630,630,630,630,630,...,1505,1505,1505,1505,1510,1510,1510,1520,1510,1510
ArrTime,901,911,920,859,918,905,944,904,903,901,...,1806,NaN,1855,NaN,NaN,NaN,1742,1831,1755,1736
CRSArrTime,915,915,915,915,915,915,915,915,915,915,...,1725,1725,1725,1725,1732,1732,1732,1744,1732,1732
UniqueCarrier,UA,UA,UA,UA,UA,UA,UA,UA,UA,UA,...,UA,UA,UA,UA,UA,UA,UA,UA,UA,UA
FlightNum,462,462,462,462,462,462,462,462,462,462,...,463,463,463,463,463,463,463,463,463,463


### Variable descriptions
        Name	         Description
    1	Year	          2004 - 2008
    2	Month	           1-12
    3	DayofMonth	       1-31
    4	DayOfWeek     	   1 (Monday) - 7 (Sunday)
    5	DepTime	           actual departure time (local, hhmm)
    6	CRSDepTime 	       scheduled departure time (local, hhmm) X
    7	ArrTime   	       actual arrival time (local, hhmm)
    8	CRSArrTime	       scheduled arrival time (local, hhmm)  X
    9	UniqueCarrier	   unique carrier code                   X        
    10	FlightNum	       flight number                         X
    11	TailNum	           plane tail number                     X
    12	ActualElapsedTime  in minutes
    13	CRSElapsedTime	   in minutes
    14	AirTime	           in minutes
    15	ArrDelay	       arrival delay, in minutes
    16	DepDelay	       departure delay, in minutes
    17	Origin	           origin IATA airport code
    18	Dest	           destination IATA airport code
    19	Distance	       in miles
    20	TaxiIn	           taxi in time, in minutes
    21	TaxiOut	           taxi out time in minutes
    22	Cancelled	       was the flight cancelled?
    23	CancellationCode   reason for cancellation (A = carrier, B = weather, C = NAS, D = security)
    24	Diverted	       1 = yes, 0 = no
    25	CarrierDelay	   in minutes
    26	WeatherDelay	   in minutes
    27	NASDelay	       in minutes
    28	SecurityDelay	   in minutes
    29	LateAircraftDelay  in minutes


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35874731 entries, 0 to 7009727
Data columns (total 29 columns):
Year                 int64
Month                int64
DayofMonth           int64
DayOfWeek            int64
DepTime              float64
CRSDepTime           int64
ArrTime              float64
CRSArrTime           int64
UniqueCarrier        object
FlightNum            int64
TailNum              object
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin               object
Dest                 object
Distance             int64
TaxiIn               float64
TaxiOut              float64
Cancelled            int64
CancellationCode     object
Diverted             int64
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
dtypes: float64(14), int64(10), object(5)
memory usage: 8.0+ GB


In [15]:
df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35874731 entries, 0 to 7009727
Data columns (total 29 columns):
Year                 35874731 non-null int64
Month                35874731 non-null int64
DayofMonth           35874731 non-null int64
DayOfWeek            35874731 non-null int64
DepTime              35194316 non-null float64
CRSDepTime           35874731 non-null int64
ArrTime              35117736 non-null float64
CRSArrTime           35874731 non-null int64
UniqueCarrier        35874731 non-null object
FlightNum            35874731 non-null int64
TailNum              35791217 non-null object
ActualElapsedTime    35114686 non-null float64
CRSElapsedTime       35872889 non-null float64
AirTime              35114686 non-null float64
ArrDelay             35114686 non-null float64
DepDelay             35194316 non-null float64
Origin               35874731 non-null object
Dest                 35874731 non-null object
Distance             35874731 non-null int64
TaxiIn       

In [16]:
df.groupby("Cancelled").size().sort_values(ascending=False)

Cancelled
0    35193128
1      681603
dtype: int64

In [17]:
cancel_delay = df.Cancelled.value_counts()
cancel_delay

0    35193128
1      681603
Name: Cancelled, dtype: int64

In [18]:
df.DepDelay.describe()

count    3.519432e+07
mean     9.619999e+00
std      3.333300e+01
min     -1.200000e+03
25%     -4.000000e+00
50%      0.000000e+00
75%      8.000000e+00
max      2.601000e+03
Name: DepDelay, dtype: float64

In [19]:
df.ArrDelay.describe()     

count    3.511469e+07
mean     8.163351e+00
std      3.649756e+01
min     -1.302000e+03
25%     -9.000000e+00
50%     -1.000000e+00
75%      1.200000e+01
max      2.598000e+03
Name: ArrDelay, dtype: float64

In [20]:
# Out of all delayed flights, percent of cancelled
if cancel_delay[0] + cancel_delay[1] > 0:
    cancelled_ratio = cancel_delay[1] / (cancel_delay[0] + cancel_delay[1])
cancelled_ratio

0.01899952922295083

## Cleaning
- Erroneous Datatype
- Manage missing values, Null values
- drop un necessary columns: 'CRSDepTime', 'CRSArrTime','TailNum','FlightNum', 'TaxiIn', 'TaxiOut','ActualElapsedTime','CRSElapsedTime','AirTime'

In [23]:
# drop un necessary columns
df.drop(['CRSDepTime', 'CRSArrTime','TailNum', 'FlightNum', 'TaxiIn','TaxiOut', 'ActualElapsedTime','CRSElapsedTime','AirTime'], axis=1, inplace=True)

### Checking

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35874731 entries, 0 to 7009727
Data columns (total 21 columns):
Year                 int64
Month                int64
DayofMonth           int64
DayOfWeek            int64
DepTime              float64
ArrTime              float64
UniqueCarrier        object
ArrDelay             float64
DepDelay             float64
Origin               object
Dest                 object
Distance             int64
Cancelled            int64
CancellationCode     object
Diverted             int64
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
day                  int64
dtypes: float64(9), int64(8), object(4)
memory usage: 5.9+ GB


In [21]:
# create a day column and assign the value 1 to the entire column
df['day'] = 1
# create a date column from the year, month, and day columns
df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
# drop the year, month, and day columns
df.drop(columns=['year', 'month', 'day'], inplace=True)

KeyError: "['year', 'month'] not in index"

## Analysing